In [1]:
import argparse
import requests

from sys import argv
from bs4 import BeautifulSoup, NavigableString, Tag
from math import ceil 
import string

import pandas as pd

from tqdm import tqdm

## Functions

In [2]:
def load_arguments(parser):
    concepto = vars(parser.parse_args(argv[1:]))["c"]
    return concepto

In [5]:
def search_medline_enciclopedia(concepto):
    
    """Search concepts in medline by letter of the alphabet"""

    # URL de origen que se 
    url = "https://medlineplus.gov/spanish/ency/encyclopedia_{}.htm".format(concepto)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    lista_articulos = list()
    for a in soup.find_all('ul', id="index"):
        for articulo in a.find_all('li'):
            lista_articulos.append(articulo.find('a', href=True)['href'])
            
    #Bucle en el que se mete en cada articulo y obtiene el abstract
    informacion = list()
    for url_articulo in lista_articulos:
        page = requests.get("https://medlineplus.gov/spanish/ency/" + url_articulo)
        soup = BeautifulSoup(page.content, 'html.parser')

        try:
            li1 = soup.find('h1', {"class": "with-also"}).text.replace("\n", " ")
            li2 = soup.find('div', {"class": "main-single"}).find('div', {'id':'ency_summary'}).text.replace("\n", " ")
            li3 = soup.find('div', {"class": "main-single"}).find('div',{'class':'section'}).text.replace("\n", " ")
            li = li1 + " "+ li2 + " " + li3
            informacion.append(li)

        except AttributeError:
            pass

    print("Number of abstracts obtained: {} for the concept: {}".format(len(informacion), concepto))
    return informacion

In [6]:
def medline_article_text(article_page):
    article_soup = BeautifulSoup(article_page.content, 'html.parser')
    article_text = ''
    summary = article_soup.find(attrs={'id': 'ency_summary'})
    if summary:
        article_text += summary.text + '\n'

    for section in article_soup.find_all('section'):
        article_text += '\n'.join([p.text for p in section.find_all('p')])
    return article_text


def medline_page(url_page, concept):
    page = requests.get(url_page)
    soup = BeautifulSoup(page.content, 'html.parser') 
    articles = []
    for article in soup.find(attrs={'class': 'results'}).find_all(attrs={'class': 'title'}):
        found = False
        article_url = 'https://vsearch.nlm.nih.gov'  + article['href']
        article_page = requests.get(article_url)

        if article_page.url.startswith('https://medlineplus.gov/'):
            article_text = medline_article_text(article_page)
            found = True

        if found:
            articles.append({'url': article_page.url, 'text': article_text, 'concept': concept})
    return articles


def medline_search(concepto):
    
    """Scrapping Medline by concept"""

    # URL de origen que se 
    url = 'https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus-spanish&v%3Asources=medlineplus-spanish-bundle&query="{}"'.format(concepto)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    num_registros = soup.find('span', class_="intronum").text.replace(",", "")
    print("Numero de articulos: {}".format(num_registros))
    
    pages = [url]
    soup_list = soup.find(attrs={'class': 'list-more list-more-bottom'})
    if soup_list:
        pages += ['https://vsearch.nlm.nih.gov' + x['href'] for x in soup_list.find_all('a')]

    #Bucle generando una lista con todos los articulos
    articles = []
    for page_url in pages:
        page_articles = medline_page(page_url, concepto)
        articles += page_articles
    
    print("Number of abstracts obtained: {} for the concept: {}".format(len(articles), concepto))
    return articles

## Code

### Search in Medline by concept

Get the acronyms from Ibereval texts to use it as concepts

In [164]:
train_abbr = pd.read_csv("../data/ibereval_data/trainning_set/clinical_cases.abbreviations.training_set.tsv", sep = '\t')

In [165]:
train_abbr.head()

,# Document_ID,StartOffset,EndOffset,Abbreviation,Definition,Definition_lemmatized
0,S0210-48062004000500008-1,1650,1652,ml,mililitro,mililitro
1,S0210-48062004000500008-1,708,709,l,litro,litro
2,S0210-48062004000500008-1,704,707,mEq,miliequivalente,miliequivalente
3,S0210-48062004000500008-1,677,681,pCO2,presión parcial de co2,presión parcial de co2
4,S0210-48062004000500008-1,2287,2290,HLA,human leucocyte antigen,human leucocyte antiger


In [185]:
amb = train_abbr.groupby('Abbreviation')['Definition'].nunique().reset_index()
amb

,Abbreviation,Definition
0,5-FU,1
1,67Ga,1
2,99-MDP,1
3,99Tc,1
4,A,3
...,...,...
763,β-HCG,2
764,γGT,1
765,μg,2
766,μl,2


In [187]:
amb.shape

(768, 2)

Filter ambiguous acronyms

In [186]:
amb[amb['Definition'] != 1]['Abbreviation'].shape

(173,)

In [188]:
defin = amb[amb['Definition'] != 1]['Abbreviation'].values.tolist()

In [189]:
acron = train_abbr[train_abbr['Abbreviation'].isin(defin)]['Definition'].values.tolist()
acron = set(acron)

In [195]:
len(acron)

391

List of acronyms with 3 or less records in train dataset (once filtered out unambiguous acronyms)

In [12]:
acron_ya = ['adenine',
 'alanina aminotransferasa',
 'alanine transaminase',
 'american medical association',
 'amiloide p',
 'analisis inmunoabsorcion ligado enzimas',
 'anatomia patologica',
 'antigeno leucocitario humano',
 'antimitochondrial antibodies',
 'aspartato transaminasa',
 'autofluorescencia fondo',
 'basofilo',
 'beta',
 'camara anterior',
 'centimetro agua',
 'citoqueratina',
 'citoqueratina 20',
 'cluster of differentiation 10',
 'cluster of differentiation 1a',
 'cluster of differentiation 3',
 'cluster of differentiation 30',
 'coding dna',
 'coding dna sequence',
 'componente 3',
 'componente 4',
 'computerized tomography',
 'creatine kinase',
 'cuarto componente complemento',
 'cumulo diferenciacion 10',
 'cumulo diferenciacion 1a',
 'cumulo diferenciacion 3',
 'cumulo diferenciacion 30',
 'cumulo diferenciacion 4',
 'cumulo diferenciacion 8',
 'cysteine',
 'cytokeratin',
 'cytokeratin 20',
 'enzymelinked immunosorbent assay',
 'evidencia mejoria clinica',
 'fibrilacion auricular',
 'fragment crystallizable',
 'frecuencia cardiaca',
 'gama glutamil transferasa',
 'hiperautofluorescencia fondo',
 'hiperfluorescein angiography',
 'horas',
 'kilogramo s',
 'leucocito',
 'linfocito',
 'luteinizing hormone',
 'masa celular total',
 'mediumchain triglycerides',
 'mercurio',
 'milimetro mercurio',
 'minuto',
 'parada cardiorrespiratoria',
 'parathyroid hormone',
 'porphyria cutanea tarda',
 'presion parcial co2',
 'presion parcial dioxido carbono',
 'procalcitonina',
 'q time',
 'reaccion cadena polimerasa',
 'resistencia vascular sistemica',
 'respuesta viral sostenida',
 'seattle',
 'segundo',
 'sindrome nefrotico congenito',
 'sistema nervioso central',
 'tercer componente complemento',
 'tercera vertebra cervical',
 'tesla',
 'tomografia computarizada',
 'tubular',
 'unidades',
 'velocidad eritrosedimentacion']

In [13]:
acron_ya = set(acron_ya)

In [14]:
len(acron_ya)

76

In [166]:
acron = train_abbr['Definition'].values.tolist()
acron = set(acron)

In [15]:
textos = []
for a in acron_ya:
    try:
        articulos = medline_search(a)
        textos += articulos
    except AttributeError:
        print('Concepts error', a)
        pass

Numero de articulos: 100
Numero de artículos obtenidos: 38 para la palabra: mercurio
Error con concepto mediumchain triglycerides
Numero de articulos: 8
Numero de artículos obtenidos: 6 para la palabra: alanina aminotransferasa
Error con concepto cluster of differentiation 1a
Error con concepto computerized tomography
Error con concepto cumulo diferenciacion 10
Error con concepto hiperautofluorescencia fondo
Numero de articulos: 7
Numero de artículos obtenidos: 7 para la palabra: procalcitonina
Error con concepto gama glutamil transferasa
Error con concepto cumulo diferenciacion 8
Error con concepto respuesta viral sostenida
Error con concepto tercera vertebra cervical
Numero de articulos: 16
Numero de artículos obtenidos: 3 para la palabra: antigeno leucocitario humano
Error con concepto presion parcial co2
Error con concepto coding dna
Numero de articulos: 1
Numero de artículos obtenidos: 1 para la palabra: cysteine
Error con concepto amiloide p
Error con concepto presion parcial dio

In [16]:
scrap2 = pd.DataFrame(textos)

In [ ]:
scrap2.head()

In [17]:
scrap2.to_csv("../data/scrapping/textos_medline_scrapping2.csv")

### Search concepts in medline by letter of the alphabet

In [97]:
search_medline_enciclopedia("A")

Numero de abstract obtenidos: 330 para la palabra: A


['Abdomen hinchado Se presenta cuando la zona ventral es más grande de lo normal. Causas  La hinchazón o distensión abdominal con frecuencia es provocada por comer en exceso más que por una enfermedad grave. El problema también puede ser causado por:Deglución de aire (un hábito nervioso)Acumulación de líquidos en el abdomen (esto puede ser un signo de un problema grave de salud)Gases en el intestino por comer alimentos ricos en fibra (como frutas y verduras)Síndrome del intestino irritableIntolerancia a la lactosaQuistes ováricosOclusión intestinal parcialEmbarazoSíndrome premenstrual (SPM)Miomas uterinosAumento de peso',
 'Ablación endometrial Es una cirugía o un procedimiento que se realiza para dañar el revestimiento del útero para minimizar el flujo menstrual abundante o prolongado. El revestimiento se llama endometrio. La cirugía puede realizarse en el hospital, en un centro de cirugía ambulatoria, o en el consultorio del proveedor. Descripción  La ablación endometrial es un proce

Search by all alphabet letters and store it in a csv file

In [98]:
enciclopedia = []
alpha = list(string.ascii_uppercase)
for letra in alpha:
    enciclopedia.append(search_medline_enciclopedia(letra))

Numero de abstract obtenidos: 330 para la palabra: A
Numero de abstract obtenidos: 86 para la palabra: B
Numero de abstract obtenidos: 350 para la palabra: C
Numero de abstract obtenidos: 217 para la palabra: D
Numero de abstract obtenidos: 457 para la palabra: E
Numero de abstract obtenidos: 82 para la palabra: F
Numero de abstract obtenidos: 67 para la palabra: G
Numero de abstract obtenidos: 146 para la palabra: H
Numero de abstract obtenidos: 260 para la palabra: I
Numero de abstract obtenidos: 2 para la palabra: J
Numero de abstract obtenidos: 2 para la palabra: K
Numero de abstract obtenidos: 96 para la palabra: L
Numero de abstract obtenidos: 151 para la palabra: M
Numero de abstract obtenidos: 79 para la palabra: N
Numero de abstract obtenidos: 54 para la palabra: O
Numero de abstract obtenidos: 293 para la palabra: P
Numero de abstract obtenidos: 21 para la palabra: Q
Numero de abstract obtenidos: 233 para la palabra: R
Numero de abstract obtenidos: 314 para la palabra: S
Nume

In [99]:
flat_enciclop = [item for sublist in enciclopedia for item in sublist]
df = pd.DataFrame(flat_enciclop)
df.to_csv('../data/scrapping/enciclopedia.csv', index=False)